In [25]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [40]:
urlpage = "http://sysbio.gzzoc.com/rpfdb/browse.html#@Scerevisiae@GSE100626"
driver = webdriver.Firefox()
try:
    driver.get(urlpage)
except TimeoutException:
    time.sleep(5)
    driver.get(urlpage)

dic = {}
tabs = driver.find_elements_by_xpath("//ul[@id = 'ScerevisiaeStudyNav']/li[contains(@class, 'tab')]/a")

In [44]:
for tab in tabs:
    link = tab.get_attribute("href")
    ActionChains(driver).move_to_element(tab).send_keys(Keys.ENTER).perform()
    tab.click()
    time.sleep(3)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ScerevisiaeDetail")))
    # WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located)
    name = tab.text
    if name != driver.find_element_by_xpath(
        "/html/body/div[2]/div/div[2]/div[1]/div[22]/div/div[@id='Scerevisiae_{}_Detail']/table/tbody/tr[2]/td[2]/a".format(name)).text:
        tab.click()
        time.sleep(3)

    dic[name] = {}
    dic[name]["pmid"] = driver.find_element_by_xpath(
        "/html/body/div[2]/div/div[2]/div[1]/div[22]/div/div[@id='Scerevisiae_{}_Detail']/table/tbody/tr[2]/td[4]/a".format(name)).get_attribute(
        "href")
    dic[name]["accession"] = driver.find_element_by_xpath(
        "/html/body/div[2]/div/div[2]/div[1]/div[22]/div/div[@id='Scerevisiae_{}_Detail']/table/tbody/tr[2]/td[2]/a".format(name)).get_attribute(
        "href")
    dic[name]["title"] = driver.find_element_by_xpath("//div[@id='Scerevisiae_{}_Detail']/table/tbody/tr[1]/td[@colspan ='3']/a".format(name)).text
    # dic[name]["strain"] = driver.find_element_by_xpath("/html/body/div[2]/div/div[2]/div[1]/div[22]/div/div[@class='tab-pane active']/table/tbody/tr[3]/td[2]").text.split(":")[1]
    strain1 = driver.find_element_by_xpath(
        "/html/body/div[2]/div/div[2]/div[1]/div[22]/div/div[@id='Scerevisiae_{}_Detail']/table/tbody/tr[3]/td[2]".format(name).format(name)).text
    strain2 = strain1.split(":")
    try:
        strain3 = strain2[1]
    except IndexError:
        strain3 = strain2[0]
    dic[name]["strain"] = strain3



In [ ]:
# driver = webdriver.Firefox()
for GSE in dic:
    url = dic[GSE]["accession"]
    # pmid = dic[GSE]["pmid"]
    # strain = dic[GSE]["strain"]
    try:
        driver.get(url)
    except TimeoutException:
        time.sleep(5)
        driver.get(url)
    if driver.find_element_by_tag_name("body").get_attribute("id") == "ui-ncbiexternallink-5":
        try:
            url = driver.find_element_by_xpath(
                "//div[@class='rprt']/div[2]/p/a[text()='{}']".format(dic[GSE]["title"])).get_attribute("href")
        except NoSuchElementException:
            try:
                url = driver.find_element_by_xpath("//div[@class='rprt']/div[2]/div[@class = 'supp']/p[@class = 'desc'][text()='{}']//ancestor::div[@class = 'rslt']//child::p/a".format(dic[GSE]["title"])).get_attribute("href")
            except:
                url = "To look" + GSE
                continue

    elif driver.find_element_by_tag_name("body").get_attribute("id") == "ui-ncbiexternallink-4":
        dic[GSE]["links"] = []
        continue
    if url == "Not found":
        dic[GSE]["pubmed"] = "Not found"
        dic[GSE]["desc"] = "Not found"
        dic[GSE]["desc2"] = "Not found"
    else:
        try:
            driver.get(url)
        except TimeoutException:
            time.sleep(5)
            driver.get(url)
        try:
            dic[GSE]["pubmed"] = driver.find_element_by_id("PubMed").get_attribute("href")
        except NoSuchElementException:
            dic[GSE]["pubmed"] = "not found"
        try:
            dic[GSE]["desc"] = driver.find_element_by_css_selector(".Title > h3:nth-child(2)").text
        except NoSuchElementException:
            dic[GSE]["desc"] = driver.find_element_by_css_selector(".Title > h2:nth-child(1)").text
        results = driver.find_element_by_id("SRA Experiments").get_attribute("href")
        driver.get(results)
        try:
            driver.find_element_by_css_selector(
                "div.results_settings:nth-child(1) > ul:nth-child(1) > li:nth-child(2) > a:nth-child(1)").click()
            WebDriverWait(driver, 100000).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "input[type='radio'][value='200']")))
            driver.find_element_by_css_selector("input[type='radio'][value='200']").click()
        except NoSuchElementException :
            print("One page ? " + results)
            # nr_pages = driver.find_element_by_id("pageno").get_attribute("last")
            # print(nr_pages)
        next_page = driver.find_elements_by_css_selector("div.pagination:nth-child(1) > a:nth-child(4)")
        dic[GSE]["links"] = []
        try:
            dic[GSE]["links"] = dic[GSE]["links"] + [x.get_attribute("href") for x in driver.find_elements_by_xpath("//*[contains(@class, 'title')]/a")]
            while (len(next_page) > 0):
                next_page[0].click()
                dic[GSE]["links"] = dic[GSE]["links"] + [x.get_attribute("href") for x in
                                 driver.find_elements_by_xpath("//*[contains(@class, 'title')]/a")]
                next_page = driver.find_elements_by_css_selector("div.pagination:nth-child(1) > a:nth-child(4)")
        except NoSuchElementException:
            dic[GSE]["links"] = [url]


In [46]:
with open("SRA_parser.tab","w") as parser:
    parser.write("Accession\tAccession link\tPMID\tStrain\tPubmed\tdesc\tdesc2\tSRA experiments link\tSRA\n")
    for GSE in dic:
        for sra_link in dic[GSE]["links"]:
            if not sra_link:
                continue
            try:
                driver.get(sra_link)
            except TimeoutException:
                time.sleep(5)
                driver.get(sra_link)
            sra = [x.text for x in
                   driver.find_elements_by_xpath("//a[contains(@href, 'trace.ncbi.nlm.nih.gov/Traces/sra/?run=')]")]
            try:
                desc2 = \
                driver.find_elements_by_css_selector("div.sra-full-data:nth-child(3) > span:nth-child(1)")[
                    0].text.split("\n")[0]
            except IndexError:
                desc2 = "to look"
            for i in sra:
                parser.write(
                    "{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(GSE,dic[GSE]["accession"],dic[GSE]["pmid"],
                                                                  dic[GSE]["strain"] ,dic[GSE]["pubmed"],dic[GSE]["desc"],
                                                                  desc2,sra_link,i))

KeyError: 'links'